# Zero-shot

In [1]:
#!pip install transformers[sentencepiece]
#!pip install 'accelerate>=0.26.0'

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch, string


ModuleNotFoundError: No module named 'transformers'

### 1. Definir etiquetas


In [3]:
etiquetas = [
    "Hostelería y Turismo",
    "Sanitario y Salud",
    "Educación y Formación",
    "Tecnología y Telecomunicaciones",
    "Construcción e Inmobiliaria",
    "Industria y Manufactura",
    "Comercio y Ventas",
    "Logística y Transporte",
    "Administración y Finanzas",
    "Cultura, Arte y Ocio"
]

In [4]:
df = pd.read_csv("EURES_LIMPIO.csv", sep=";") 
df.head()

,id,timestamp,titulo,ocupacion,descripcion,provincia,tipo_contrato,descripcion_proc
0,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,AGENTE COMERCIAL DE SEGUROS (REF.: 6891),corredor de seguros/corredora de seguros,tareas:prospección de nuevos asegurados.planif...,Asturias,Contrato,tarea prospección asegurado planificación gest...
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PERSONAL CONDUCCIÓN DE CAMIONES RÍGIDOS Y GÓND...,Conductor de vehículo de carga/conductora de v...,descripción: se necesita cubrir cuatro puestos...,Huesca,Contrato,descripción necesitar cubrir puesto empresa mo...
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,EDUCADORES SOCIALES,Trabajador social/trabajadora social,educador social para hogar en arinaga. fines d...,Las Palmas,Determinado,educador social hogar arinaga fin semana festi...
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PIZZERO (REF. 042025002051),Pizzero/pizzera,funciones: elaboración de pizzas requisitos: 2...,Islas Baleares,Determinado,función elaboración pizza requisito mes experi...
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,INTÉRPRETES DE LA LENGUA DE SIGNOS,intérprete de lengua de signos,intérprete de lengua de signos para puestos en...,Santa Cruz de Tenerife,Determinado,intérprete lengua signo puesto ies manuel mart...


### 3. Inicializar modelos

In [11]:
clf1 = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=0
)

clf2 = pipeline(
    "zero-shot-classification",
    model="Recognai/zeroshot_selectra_medium",
    device=0
)

tokenizer3 = AutoTokenizer.from_pretrained("DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
model3 = AutoModelForSequenceClassification.from_pretrained(
    "DAMO-NLP-SG/zero-shot-classify-SSTuning-XLM-R")
list_ABC = [x for x in string.ascii_uppercase]

def classify_model3(text, labels):
    labels = [x+'.' if not x.endswith('.') else x for x in labels]
    labels_new = labels + [tokenizer3.pad_token] * (20 - len(labels))
    s_option = ' '.join([f'({list_ABC[i]}) {labels_new[i]}' for i in range(len(labels_new))])
    text_input = f'{s_option} {tokenizer3.sep_token} {text}'
    encoding = tokenizer3([text_input], truncation=True, max_length=512, return_tensors='pt')
    # No movemos a device manualmente, lo gestiona accelerate
    logits = model3(**encoding).logits
    logits = logits[:, :len(labels)]
    probs = torch.nn.functional.softmax(logits, dim=-1).tolist()[0]
    return list(zip([lab.strip('.') for lab in labels], probs))

Device set to use cuda:0
Device set to use cuda:0


### 4. Voting classifier


In [12]:
def classify_offer(text):
    # Modelo 1
    out1 = clf1(text, etiquetas, hypothesis_template="Este texto es sobre {}")
    probs1 = {lab: score for lab, score in zip(out1["labels"], out1["scores"])}

    # Modelo 2
    out2 = clf2(text, candidate_labels=etiquetas, hypothesis_template="Este ejemplo es {}.")
    probs2 = {lab: score for lab, score in zip(out2["labels"], out2["scores"])}

    # Modelo 3
    results3 = classify_model3(text, etiquetas)  # <-- modificado para devolver lista (label, prob)
    probs3 = {lab.strip('.'): prob for lab, prob in results3}

    # Combinar: promedio de probabilidades
    combined = {}
    for lab in etiquetas:
        combined[lab] = np.mean([
            probs1.get(lab, 0),
            probs2.get(lab, 0),
            probs3.get(lab, 0)
        ])

    # Seleccionar la clase con mayor probabilidad media
    final = max(combined, key=combined.get)
    return final, combined



In [17]:
BATCH_SIZE = 8  # ajusta según tu GPU

def classify_batch(texts):
    results = []
    # Modelo 1
    out1 = clf1(texts, etiquetas, hypothesis_template="Este texto es sobre {}")
    # Modelo 2
    out2 = clf2(texts, candidate_labels=etiquetas, hypothesis_template="Este ejemplo es {}.")
    # Modelo 3 (aún fila a fila, pero podrías vectorizarlo)
    out3 = [classify_model3(t, etiquetas) for t in texts]

    for i, text in enumerate(texts):
        probs1 = {lab: score for lab, score in zip(out1[i]["labels"], out1[i]["scores"])}
        probs2 = {lab: score for lab, score in zip(out2[i]["labels"], out2[i]["scores"])}
        probs3 = {lab: prob for lab, prob in out3[i]}

        combined = {lab: np.mean([
            probs1.get(lab, 0),
            probs2.get(lab, 0),
            probs3.get(lab, 0)
        ]) for lab in etiquetas}

        final = max(combined, key=combined.get)
        results.append((final, combined))
    return results

# Aplicar en lotes con barra de progreso
sectors, probs = [], []
for i in tqdm(range(0, len(df), BATCH_SIZE)):
    batch = df["ocupacion"].iloc[i:i+BATCH_SIZE].tolist()
    batch_results = classify_batch(batch)
    for f, c in batch_results:
        sectors.append(f)
        probs.append(c)

df["sector"] = sectors
df["probs"] = probs


100%|██████████| 908/908 [2:23:21<00:00,  9.47s/it]  


### 5. Aplicar al dataset


In [ ]:

# =========================
# 6. Guardar resultados
# =========================
df.to_csv("EURES_CATEGORIZADO.csv", index=False)
df.head()


,id,timestamp,titulo,ocupacion,descripcion,provincia,tipo_contrato,descripcion_proc,sector,probs
0,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,AGENTE COMERCIAL DE SEGUROS (REF.: 6891),corredor de seguros/corredora de seguros,tareas:prospección de nuevos asegurados.planif...,Asturias,Contrato,tarea prospección asegurado planificación gest...,Administración y Finanzas,"{'Hostelería y Turismo': 0.04141047534843286, ..."
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PERSONAL CONDUCCIÓN DE CAMIONES RÍGIDOS Y GÓND...,Conductor de vehículo de carga/conductora de v...,descripción: se necesita cubrir cuatro puestos...,Huesca,Contrato,descripción necesitar cubrir puesto empresa mo...,Logística y Transporte,"{'Hostelería y Turismo': 0.03098547427604596, ..."
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,EDUCADORES SOCIALES,Trabajador social/trabajadora social,educador social para hogar en arinaga. fines d...,Las Palmas,Determinado,educador social hogar arinaga fin semana festi...,Educación y Formación,"{'Hostelería y Turismo': 0.09141705681880315, ..."
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,PIZZERO (REF. 042025002051),Pizzero/pizzera,funciones: elaboración de pizzas requisitos: 2...,Islas Baleares,Determinado,función elaboración pizza requisito mes experi...,Hostelería y Turismo,"{'Hostelería y Turismo': 0.3217993925015132, '..."
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,INTÉRPRETES DE LA LENGUA DE SIGNOS,intérprete de lengua de signos,intérprete de lengua de signos para puestos en...,Santa Cruz de Tenerife,Determinado,intérprete lengua signo puesto ies manuel mart...,"Cultura, Arte y Ocio","{'Hostelería y Turismo': 0.08064752444624901, ..."


In [20]:
df.sector.value_counts()

sector
Logística y Transporte             1291
Sanitario y Salud                  1211
Industria y Manufactura             963
Administración y Finanzas           885
Construcción e Inmobiliaria         791
Hostelería y Turismo                585
Comercio y Ventas                   542
Educación y Formación               436
Tecnología y Telecomunicaciones     283
Cultura, Arte y Ocio                273
Name: count, dtype: int64

In [6]:
df = pd.read_csv("EURES_CAT_SENTIMENT.csv")

In [17]:
import re
def clean_and_parse(s):
    if isinstance(s, str):
        # 1. Eliminar np.float64( ... )
        s = re.sub(r'np\.float64\((.*?)\)', r'\1', s)
        # 2. Convertir a dict de Python
        return ast.literal_eval(s)
    return s  # si ya es dict

# Aplicar la función a toda la columna
df["probs"] = df["probs"].apply(clean_and_parse)

# Expandir todas las claves en columnas, rellenando con 0
df_expandido = df["probs"].apply(pd.Series).fillna(0)

# Unir con el resto del DataFrame
df_final = pd.concat([df[["id", "timestamp","provincia", "titulo", "sentimiento"]], df_expandido], axis=1)

df_final.columns


Index(['id', 'timestamp', 'provincia', 'titulo', 'sentimiento',
       'Hostelería y Turismo', 'Sanitario y Salud', 'Educación y Formación',
       'Tecnología y Telecomunicaciones', 'Construcción e Inmobiliaria',
       'Industria y Manufactura', 'Comercio y Ventas',
       'Logística y Transporte', 'Administración y Finanzas',
       'Cultura, Arte y Ocio'],
      dtype='object')

In [18]:
df_final.head()

,id,timestamp,provincia,titulo,sentimiento,Hostelería y Turismo,Sanitario y Salud,Educación y Formación,Tecnología y Telecomunicaciones,Construcción e Inmobiliaria,Industria y Manufactura,Comercio y Ventas,Logística y Transporte,Administración y Finanzas,"Cultura, Arte y Ocio"
0,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,Asturias,AGENTE COMERCIAL DE SEGUROS (REF.: 6891),0,0.041410,0.120989,0.041578,0.059036,0.030720,0.061685,0.163560,0.137138,0.324149,0.019736
1,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,Huesca,PERSONAL CONDUCCIÓN DE CAMIONES RÍGIDOS Y GÓND...,0,0.030985,0.057478,0.036354,0.029181,0.147310,0.066028,0.055457,0.540594,0.018703,0.017910
2,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,Las Palmas,EDUCADORES SOCIALES,0,0.091417,0.192628,0.193099,0.050107,0.051788,0.075421,0.080538,0.079231,0.115695,0.070076
3,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,Islas Baleares,PIZZERO (REF. 042025002051),0,0.321799,0.063992,0.053844,0.034645,0.072012,0.090498,0.116909,0.045905,0.064227,0.136169
4,https://europa.eu/eures/portal/jv-se/jv-detail...,10/10/2025,Santa Cruz de Tenerife,INTÉRPRETES DE LA LENGUA DE SIGNOS,0,0.080648,0.092984,0.213390,0.103763,0.021884,0.035743,0.067119,0.039501,0.032571,0.312398


In [20]:
df_final.to_csv("EURES_NUMERICO.csv", index=False)